In [1]:
import numpy as np
from tqdm import tqdm

import mlrfit as mf
import numba as nb

from scipy.linalg import block_diag  
from mlrfit import frob_loss, rel_diff, diag_sparseBCt, LinOpResidualMatrix, compute_perm_residual

In [2]:
def test_single_level_factor_fit(R, ranks, hpart, level, symm=False, PSD=False):
    """
    Return updated block diagonal, where each block is BCt
    delta_rm1[level]: scores for decreasing a rank by 1
    delta_rp1[level]: scores for increasing a rank by 1
    """
    dim = ranks[level]
    num_blocks = len(hpart['rows']['lk'][level])-1
    r1, c1 = 0, 0
    m, n = hpart['rows']['lk'][0][-1], hpart['cols']['lk'][0][-1]
    A_level = np.zeros((m, n))
    for block in range(num_blocks):
        r1, r2 = hpart['rows']['lk'][level][block], hpart['rows']['lk'][level][block+1]
        c1, c2 = hpart['cols']['lk'][level][block], hpart['cols']['lk'][level][block+1]
        if PSD:
            assert r1==c1 and r2==c2
            U, sigmas = mf.frob_low_rank_psd(R[r1:r2, c1:c2], dim = dim+1)
            Vt = U.T
        else:
            U, Vt, sigmas = mf.frob_low_rank(R[r1:r2, c1:c2], dim = dim+1, symm=symm)
        max_rank_block = min(r2-r1, c2-c1)
        # print(sigmas)
        if max_rank_block-1 >= dim >= 1 and sigmas.size >= dim+1:
            A_level[r1:r2, c1:c2] = U[:, :-1] @ np.diag(sigmas[:-1]) @ Vt[:-1, :] 
        elif dim >= max_rank_block or sigmas.size <= dim:
            A_level[r1:r2, c1:c2] = U @ np.diag(sigmas) @ Vt
        r1 = r2; c1 = c2 
    return A_level

In [3]:
M = 10
m, n  = 100, 75
dim = 10

# Test low rank implementation
for _ in tqdm(range(M)):
    A = np.random.randn(m, n)*10
    U2, Vt2, sigmas2 = mf.frob_low_rank(A, dim=min(m,n))
    mf.test_eigsh_svds(U2, sigmas2, Vt2, min(m,n), A, mode='svds')
    assert (np.diff(sigmas2) <= 1e-9).all() and (sigmas2 == np.sort(sigmas2)[::-1]).all()
    assert np.allclose(U2 @ np.diag(sigmas2) @ Vt2, A)

for _ in tqdm(range(M)):
    A = np.random.randn(m, n)*10
    U2, Vt2, sigmas2 = mf.frob_low_rank(A, dim=dim)
    mf.test_eigsh_svds(U2, sigmas2, Vt2, dim, A, mode='svds')
    assert (np.diff(sigmas2) <= 1e-9).all() and (sigmas2 == np.sort(sigmas2)[::-1]).all()

print("PASSED low rank implementation tests")


# Test low rank and single level fit implementation
hpart = mf.random_hpartition(m,  n)
mf.test_hpartition(hpart, m, n)
num_levels = len(hpart["cols"]["lk"])
hat_A = mf.MLRMatrix(hpart=hpart, debug=True)

for _ in tqdm(range(M)):
    ranks = np.random.randint(1, min(m,n), num_levels)
    R = np.random.randn(m,n)
    for level in range(num_levels):
        B_level, C_level, delta_rm1, delta_rp1, b_level, c_level = mf.single_level_factor_fit(R, ranks, hpart, \
                                                        level)
        A_level = test_single_level_factor_fit(R, ranks, hpart, level)
        assert (delta_rm1 + 1e-9 >= delta_rp1)
        assert np.allclose(A_level, hat_A._block_diag_BCt(level, hpart, B_level, C_level))
print("PASSED single_level_factor_fit implementation tests")

# Test low rank and block coordinate descent implementation
hpart = mf.random_hpartition(m,  n)
mf.test_hpartition(hpart, m, n)
eps = 1e-2

for _ in tqdm(range(5)):
    ranks = np.random.randint(1, min(m,n), num_levels)
    A = np.random.randn(m,n)
    
    cycle_size = 1 
    losses = hat_A.factor_fit(A, ranks, hpart, eps_ff=eps, method='bcd',\
                                max_iters_ff=10**3, symm=False, warm_start=False)
    for i in range(1, len(losses)-cycle_size):
        assert losses[-i-cycle_size] - losses[-i] >= -1e-9, \
            print(f"{i = }, {losses[-i-cycle_size] - losses[-i]}", \
                "loss is not decreasing with epochs")
    assert eps >= losses[-1-cycle_size] - losses[-1]
    assert np.allclose(mf.rel_diff(hat_A.matrix(), den=A), losses[-1]),\
        print(mf.rel_diff(hat_A.matrix(), den=A), losses[-1])
    
    rows_lk = nb.typed.List(hpart['rows']['lk'])
    cols_lk = nb.typed.List(hpart['cols']['lk'])
    

print("PASSED factor_fit implementation tests")

100%|██████████| 10/10 [00:00<00:00, 155.47it/s]


PASSED low rank implementation tests


100%|██████████| 10/10 [00:02<00:00,  3.80it/s]


PASSED single_level_factor_fit implementation tests


  0%|          | 0/5 [00:00<?, ?it/s]/Users/parshakova.tanya/Documents/projects/MLR_Matrices/mlr_fitting/mlrfit/utils.py:409: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (Array(float64, 2, 'A', False, aligned=True), Array(float64, 2, 'A', False, aligned=True))
  hat_A_except_level[r1:r2, c1:c2] += np.dot(B_level[r1:r2], C_level[c1:c2].T)
100%|██████████| 5/5 [00:18<00:00,  3.66s/it]

PASSED factor_fit implementation tests


In [4]:
ranks = np.array([10, 7, 5, 4, 1])
ranks1 = ranks + 0
ranks2 = ranks * 2
n = 200

pi_rows = np.random.permutation(n)
hpart = {'rows':{'pi':pi_rows, 'lk':[]}, 'cols':{'pi':pi_rows, 'lk':[]}} 
for ngroups in [2, 5, 9, 17, n+1]:
       hpart['rows']['lk'] += [ np.linspace(0, n, ngroups, endpoint=True, dtype=int)]
hpart['rows']['lk'][1] = np.delete(hpart['rows']['lk'][1], -2)
hpart['rows']['lk'][2] = np.delete(hpart['rows']['lk'][2], -4)
hpart['cols']['lk'] = hpart['rows']['lk']


for _ in range(10):
    B1, C1 = np.random.randn(n, ranks1.sum()), np.random.randn(n, ranks1.sum())
    B2, C2 = np.random.randn(n, ranks2.sum()), np.random.randn(n, ranks2.sum())

    mlr1 = mf.MLRMatrix(B=B1, C=C1, hpart=hpart, ranks=ranks1)
    mlr1.construct_sparse_format()

    mlr2 = mf.MLRMatrix(B=B2, C=C2, hpart=hpart, ranks=ranks2)
    mlr2.construct_sparse_format()
    
    B, C, ranks = mf.mlr_mlr_symm_hpar_matmul(B1, C1, ranks1, B2, C2, ranks2, hpart["rows"]["lk"])

    true_AA_p = mlr1.matrix() @ mlr2.matrix()
    mlr_prod = mf.MLRMatrix(B=B, C=C, hpart=hpart, ranks=ranks)

    assert np.allclose(true_AA_p, mlr_prod.matrix())
    assert np.allclose(np.diag(true_AA_p), diag_sparseBCt(B, C, hpart['rows']['lk'], ranks)[mlr_prod.pi_inv_rows])

print("PASSED")

PASSED


In [5]:
for _ in range(20):
    B1, C1 = np.random.randn(n, ranks1.sum()), np.random.randn(n, ranks1.sum())

    mlr1 = mf.MLRMatrix(B=B1, C=C1, hpart=hpart, ranks=ranks1)
    mlr1.construct_sparse_format()

    mlr2 = mf.MLRMatrix(B=C1, C=B1, hpart=hpart, ranks=ranks1)
    mlr2.construct_sparse_format()
    
    B, C, ranks = mf.mlr_mlr_symm_hpar_matmul(B1, C1, ranks1, C1, B1, ranks1, hpart["rows"]["lk"])

    true_AA_p = mlr1.matrix() @ mlr2.matrix()
    mlr_prod = mf.MLRMatrix(B=B, C=C, hpart=hpart, ranks=ranks)

    assert np.allclose(true_AA_p, mlr_prod.matrix())
    assert np.allclose(np.diag(true_AA_p), diag_sparseBCt(B, C, hpart['rows']['lk'], ranks)[mlr_prod.pi_inv_rows])

    G = np.random.randn(n, n//2)
    perm_A = G @ G.T
    rows_lk = hpart['rows']['lk']
    cols_lk = hpart['cols']['lk']

    l1 = frob_loss(perm_A, B, C, rows_lk, cols_lk, ranks)
    l2 = frob_loss((G, G), B, C, rows_lk, cols_lk, ranks)
    assert np.allclose(l1, l2)

print("PASSED frob_loss, matmul and diag_sparseBCt")

PASSED frob_loss, matmul and diag_sparseBCt


In [6]:
n  = 300
dim = 10

# Test low rank and single level fit implementation
hpart = mf.random_hpartition(n,  n, symm=True)
mf.test_hpartition(hpart, n, n)
num_levels = len(hpart['rows']['lk']) 
hat_A = mf.MLRMatrix(hpart=hpart, debug=True)
rows_lk = hpart['rows']['lk']
cols_lk = hpart['cols']['lk']

for _ in tqdm(range(20)):
    for PSD in [True, False]:
        ranks = np.random.randint(1, n//3, num_levels)
        ranks[-1] = 1
        G = np.random.randn(n, n//3)
        perm_A = G @ G.T
        B, C = np.random.randn(n, ranks.sum()), np.random.randn(n, ranks.sum())
        if PSD: C = B

        l1 = frob_loss(perm_A, B, C, rows_lk, cols_lk, ranks)
        l2 = frob_loss((G, G), B, C, rows_lk, cols_lk, ranks)
        assert np.allclose(l1, l2)
        
        levels = np.concatenate([np.arange(num_levels), np.arange(num_levels-1)[::-1]], axis=0)
        for level in levels:
            R = compute_perm_residual(perm_A, B, C, level, rows_lk, cols_lk, ranks)
            R2 = compute_perm_residual((G, G), B, C, level, rows_lk, cols_lk, ranks)
            assert np.allclose(R, R2.toarray()), print(rel_diff(R, R2.toarray()))
            B_level, C_level, delta_rm1, delta_rp1, b_level, c_level = mf.single_level_factor_fit(R, ranks, hpart, \
                                                            level, PSD=PSD)
            A_level = test_single_level_factor_fit(R, ranks, hpart, level, PSD=PSD)
            assert (delta_rm1 + 1e-9 >= delta_rp1)
            assert np.allclose(A_level, hat_A._block_diag_BCt(level, hpart, B_level, C_level))
            # A_level = hat_A._block_diag_BCt(level, hpart, B_level, C_level)
            B_level, C_level, delta_rm1, delta_rp1, b_level, c_level = mf.single_level_factor_fit(R2, ranks, hpart, \
                                                            level, PSD=PSD)
            assert (delta_rm1 + 1e-9 >= delta_rp1)
            assert np.allclose(A_level, hat_A._block_diag_BCt(level, hpart, B_level, C_level))

print("PASSED compute_perm_residual implementation tests")

100%|██████████| 20/20 [01:43<00:00,  5.19s/it]

PASSED compute_perm_residual implementation tests


In [7]:
m, n  = 150, 100
dim = 10
PSD = False

# Test low rank and single level fit implementation
hpart = mf.random_hpartition(m,  n)
mf.test_hpartition(hpart, m, n)
num_levels = len(hpart['rows']['lk']) 
hat_A = mf.MLRMatrix(hpart=hpart, debug=True)
rows_lk = hpart['rows']['lk']
cols_lk = hpart['cols']['lk']

for _ in tqdm(range(10)):
    ranks = np.random.randint(1, n//2, num_levels)
    ranks[-1] = 1
    G1 = np.random.randn(m, n//3)
    G2 = np.random.randn(n, n//3)
    perm_A = G1 @ G2.T
    B, C = np.random.randn(m, ranks.sum()), np.random.randn(n, ranks.sum())
    
    levels = np.concatenate([np.arange(num_levels), np.arange(num_levels-1)[::-1]], axis=0)
    for level in levels:
        R = compute_perm_residual(perm_A, B, C, level, rows_lk, cols_lk, ranks)
        R2 = compute_perm_residual((G1, G2), B, C, level, rows_lk, cols_lk, ranks)
        assert np.allclose(R, R2.toarray()), print(rel_diff(R, R2.toarray()))
        B_level, C_level, delta_rm1, delta_rp1, b_level, c_level = mf.single_level_factor_fit(R, ranks, hpart, \
                                                        level, PSD=PSD)
        A_level = test_single_level_factor_fit(R, ranks, hpart, level, PSD=PSD)
        assert (delta_rm1 + 1e-9 >= delta_rp1)
        assert np.allclose(A_level, hat_A._block_diag_BCt(level, hpart, B_level, C_level))
        B_level, C_level, delta_rm1, delta_rp1, b_level, c_level = mf.single_level_factor_fit(R2, ranks, hpart, \
                                                        level, PSD=PSD)
        assert (delta_rm1 + 1e-9 >= delta_rp1)
        assert np.allclose(A_level, hat_A._block_diag_BCt(level, hpart, B_level, C_level))

print("PASSED compute_perm_residual implementation tests")

100%|██████████| 10/10 [00:03<00:00,  2.61it/s]

PASSED compute_perm_residual implementation tests
